In [0]:
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout
import warnings
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.engine.topology import get_source_inputs
from keras.utils import get_file
from keras.utils import layer_utils


sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Modular function for Fire Node

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


# Original SqueezeNet from paper.

def SqueezeNet(include_top=True, weights='imagenet',
               input_tensor=None, input_shape=None,
               pooling=None,
               classes=1000):
    """Instantiates the SqueezeNet architecture.
    """
        
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')


    input_shape = _obtain_input_shape(input_shape,
                                      default_size=227,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor


    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    if include_top:
        # It's not obvious where to cut the network... 
        # Could do the 8th or 9th layer... some work recommends cutting earlier layers.
    
        x = Dropout(0.5, name='drop9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = Activation('relu', name='relu_conv10')(x)
        x = GlobalAveragePooling2D()(x)
        x = Activation('softmax', name='loss')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling=='max':
            x = GlobalMaxPooling2D()(x)
        elif pooling==None:
            pass
        else:
            raise ValueError("Unknown argument for 'pooling'=" + pooling)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='squeezenet')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
            
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model


In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive

In [0]:
import tensorflow as tf
import keras
import os
import numpy
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dense
from keras.layers import Dropout

In [0]:
train_datagen= ImageDataGenerator()
valid_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(

    directory='PATH/TO/TRAIN/DATASET',

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)


valid_generator = valid_datagen.flow_from_directory(

    directory='PATH/TO/VALID/DATASET',

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)

test_generator= test_datagen.flow_from_directory(

   directory='PATH/TO/VALID/DATASET',

   target_size=(224,224),color_mode="rgb",

   class_mode="categorical",shuffle=False,seed=42)


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [0]:
base_model = SqueezeNet(include_top=False, weights='imagenet',input_tensor=None,input_shape=(224,224,3),pooling=None,classes=1000)

x = base_model.output
x = Dropout(0.5, name='drop9')(x)
x = Convolution2D(256, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 111, 111, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 111, 111, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 55, 55, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [0]:
model.fit_generator(generator=train_generator,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,steps_per_epoch=STEP_SIZE_TRAIN,epochs=100)

W0709 12:27:20.277752 140407586551680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
126/126 [==============================] - 1397s 11s/step - loss: 1.6292 - acc: 0.3425 - val_loss: 0.9762 - val_acc: 0.5411
Epoch 2/100
126/126 [==============================] - 65s 518ms/step - loss: 1.0043 - acc: 0.5429 - val_loss: 0.8969 - val_acc: 0.6253
Epoch 3/100
126/126 [==============================] - 65s 513ms/step - loss: 0.8721 - acc: 0.6099 - val_loss: 0.9630 - val_acc: 0.6085
Epoch 4/100
126/126 [==============================] - 65s 517ms/step - loss: 0.7028 - acc: 0.7110 - val_loss: 0.8910 - val_acc: 0.6764
Epoch 5/100
126/126 [==============================] - 65s 516ms/step - loss: 0.7236 - acc: 0.6997 - val_loss: 0.7335 - val_acc: 0.7150
Epoch 6/100
126/126 [==============================] - 65s 518ms/step - loss: 0.5986 - acc: 0.7716 - val_loss: 0.5888 - val_acc: 0.7863
Epoch 7/100
126/126 [==============================] - 65s 513ms/step - loss: 0.5335 - acc: 0.7956 - val_loss: 0.5733 - val_acc: 0.7829
Epoch 8/100
126/126 [===========================

In [0]:
#EVALUATE the model
model.evaluate_generator(test_generator,steps = STEP_SIZE_TEST, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

[0.13583221323654016, 0.96875]

In [0]:
#saving the model
model.save("PATH/TO/SAVE/MODEL/SqueezeNets.model")

##Converting to TfLite

In [0]:
import tensorflow as tf
import numpy as np

converter  = tf.lite.TFLiteConverter.from_keras_model_file("PATH/TO/SAVED/MODEL/SqueezeNets.model", input_shapes = {'input_1' : [1,224,224,3]})
tflite_model  = converter.convert()

open("PATH/TO/SAVE/MODEL/SqueezeNetS.tflite", "wb").write(tflite_model)

##Getting the F1 scores

In [0]:
import os
validation_data_dir = "PATH/TO/DATASET"
import glob
allvalidimgpaths=glob.glob(os.path.join(validation_data_dir, '*/*.jpeg'))
allvalidimgpaths+=glob.glob(os.path.join(validation_data_dir, '*/*.jpg'))
import random
random.shuffle(allvalidimgpaths)

In [0]:
from keras.preprocessing.image import load_img, img_to_array
validation_imgs = [img_to_array(load_img(img, target_size=(224,224))) for img in allvalidimgpaths]

In [0]:
validation_labels= [fn.split('/')[-2] for fn in allvalidimgpaths]

import numpy as np
np.array(validation_imgs)

print(np.shape(validation_imgs))

(221, 224, 224, 3)


In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#LABELS
le.fit(["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"])
validation_labels_enc = le.transform(validation_labels)

In [0]:
print(validation_labels[5:9])
print(validation_labels_enc[5:9])

['chana dal', 'masoor dal', 'kidney beans', 'toor dal']
[1 3 2 5]


In [0]:
from keras.models import load_model
model = load_model("PATH/TO/SAVED/MODEL/SqueezeNets.model")

In [0]:
import numpy
newvalids=validation_imgs[:]

valid_imgs_array=numpy.asarray(validation_imgs)

print(valid_imgs_array.shape)
valid_imgs_array.astype('float32')

#valid_imgs_array/=255

test_predictions=model.predict(valid_imgs_array)


In [0]:
import numpy as np
newpreds= np.argmax(test_predictions,axis=1)

from sklearn.metrics import classification_report

print(classification_report(validation_labels_enc, newpreds, target_names=["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"]))

                  precision    recall  f1-score   support

black eyed beans       0.97      1.00      0.99        34
       chana dal       0.69      0.85      0.76        13
    kidney beans       1.00      0.96      0.98        75
      masoor dal       1.00      0.99      0.99        70
       moong dal       0.67      1.00      0.80         4
        toor dal       1.00      0.88      0.93        16
        urad dal       0.89      0.89      0.89         9

        accuracy                           0.96       221
       macro avg       0.89      0.94      0.91       221
    weighted avg       0.97      0.96      0.96       221

